# 03. Atlas Application and ROI Definition

This notebook loads the LC atlas, defines a reference region, and visualizes both on registered data.

## Atlas Choice

We use the **Ye et al. (2021) 7T probabilistic LC atlas** because:
- Derived from 7T data (matches our target field strength)
- Probabilistic (allows thresholding flexibility)
- Publicly available on NITRC

Alternative: Keren et al. (2009) 3T atlas—we may compare both for sensitivity analysis.

In [ ]:
import sys
sys.path.append('../')
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn import plotting, image

from src.io import load_nifti

ATLAS_DIR = '../atlases'
OUTPUT_DIR = '../outputs/results'
FIGURES_DIR = '../outputs/figures'
DATA_DIR = '../data'

os.makedirs(FIGURES_DIR, exist_ok=True)

## 1. Load LC Atlas

Download from: https://www.nitrc.org/projects/lc_7t_prob

In [ ]:
# List available atlases
print("Available atlas files:")
for f in os.listdir(ATLAS_DIR):
    if f.endswith('.nii.gz') or f.endswith('.nii'):
        print(f"  {f}")

In [ ]:
# Load LC atlas (update filename as needed)
LC_ATLAS_FILE = 'LC_prob_MNI.nii.gz'  # Adjust to actual filename
lc_atlas_path = os.path.join(ATLAS_DIR, LC_ATLAS_FILE)

if os.path.exists(lc_atlas_path):
    lc_atlas_img = nib.load(lc_atlas_path)
    lc_atlas_data = lc_atlas_img.get_fdata()
    
    print(f"LC Atlas loaded: {LC_ATLAS_FILE}")
    print(f"  Shape: {lc_atlas_data.shape}")
    print(f"  Affine:\n{lc_atlas_img.affine}")
    print(f"  Value range: [{lc_atlas_data.min():.3f}, {lc_atlas_data.max():.3f}]")
    print(f"  Non-zero voxels: {np.sum(lc_atlas_data > 0)}")
else:
    print(f"Atlas not found at: {lc_atlas_path}")
    print("Please download from NITRC and place in atlases/ directory.")
    lc_atlas_img = None

## 2. Visualize Atlas

Show the LC atlas location in MNI space. The LC is located in the dorsal pons, adjacent to the 4th ventricle.

In [ ]:
if lc_atlas_img is not None:
    # Plot atlas on MNI template
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Get MNI template for background
    from nilearn.datasets import load_mni152_template
    mni_template = load_mni152_template()
    
    # LC is approximately at these MNI coordinates
    # x: ±4-6mm (bilateral), y: -35 to -40mm, z: -25 to -35mm
    
    for i, (display_mode, cut_coords, title) in enumerate([
        ('z', [-30], 'Axial (z=-30)'),
        ('y', [-37], 'Coronal (y=-37)'),
        ('x', [5], 'Sagittal (x=5)')
    ]):
        display = plotting.plot_roi(
            lc_atlas_img,
            bg_img=mni_template,
            cut_coords=cut_coords,
            display_mode=display_mode,
            axes=axes[i],
            title=title,
            alpha=0.7,
            cmap='autumn'
        )
    
    plt.suptitle('LC Atlas in MNI Space (Ye et al. 7T)', y=1.02, fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, 'LC_atlas_MNI.png'), dpi=150, bbox_inches='tight')
    plt.show()

## 3. Create Binary Mask at Different Thresholds

The probabilistic atlas assigns each voxel a probability of being LC. We examine different thresholds.

In [ ]:
if lc_atlas_img is not None:
    thresholds = [0.1, 0.25, 0.5, 0.75]
    
    print("LC ROI size at different probability thresholds:")
    print("-" * 40)
    
    for thresh in thresholds:
        mask = lc_atlas_data > thresh
        n_voxels = np.sum(mask)
        
        # Estimate volume (depends on voxel size)
        voxel_dims = lc_atlas_img.header.get_zooms()[:3]
        voxel_vol_mm3 = np.prod(voxel_dims)
        volume_mm3 = n_voxels * voxel_vol_mm3
        
        print(f"  Threshold {thresh:.2f}: {n_voxels:4d} voxels ({volume_mm3:.1f} mm³)")
    
    print("\nNote: Anatomical LC is approximately 2mm × 2mm × 15mm ≈ 60 mm³")

## 4. Define Reference Region

For CNR calculation, we need a reference region in the pontine tegmentum adjacent to LC.

Options:
1. Use an existing brainstem atlas
2. Define a simple geometric ROI
3. Use a published reference region from LC literature

In [ ]:
def create_pontine_reference_roi(shape, affine, lc_mask=None):
    """
    Create a simple pontine tegmentum reference region.
    
    Located ventral and medial to LC, in the central pons.
    MNI coordinates approximately: x=0, y=-30, z=-30
    """
    # Create empty mask in same space
    ref_mask = np.zeros(shape, dtype=np.float32)
    
    # Convert MNI coordinates to voxel indices
    # Reference region: sphere centered at (0, -30, -28)
    center_mni = np.array([0, -30, -28, 1])
    inv_affine = np.linalg.inv(affine)
    center_vox = inv_affine @ center_mni
    cx, cy, cz = int(center_vox[0]), int(center_vox[1]), int(center_vox[2])
    
    # Create spherical ROI (radius ~3mm = ~3 voxels at 1mm)
    radius = 3
    for x in range(max(0, cx-radius), min(shape[0], cx+radius+1)):
        for y in range(max(0, cy-radius), min(shape[1], cy+radius+1)):
            for z in range(max(0, cz-radius), min(shape[2], cz+radius+1)):
                if (x-cx)**2 + (y-cy)**2 + (z-cz)**2 <= radius**2:
                    ref_mask[x, y, z] = 1.0
    
    # Exclude LC voxels if provided
    if lc_mask is not None:
        ref_mask[lc_mask > 0] = 0
    
    return ref_mask

if lc_atlas_img is not None:
    # Create reference region
    lc_binary = lc_atlas_data > 0.1  # Exclude any LC probability
    ref_mask = create_pontine_reference_roi(
        lc_atlas_data.shape,
        lc_atlas_img.affine,
        lc_binary
    )
    
    ref_img = nib.Nifti1Image(ref_mask, lc_atlas_img.affine)
    
    # Save reference region
    ref_path = os.path.join(ATLAS_DIR, 'pontine_reference_MNI.nii.gz')
    nib.save(ref_img, ref_path)
    
    print(f"Reference region created: {np.sum(ref_mask > 0)} voxels")
    print(f"Saved to: {ref_path}")

In [ ]:
# Visualize LC and reference together
if lc_atlas_img is not None:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Combine LC and reference into one image for visualization
    combined = np.zeros_like(lc_atlas_data)
    combined[lc_atlas_data > 0.5] = 1  # LC in red
    combined[ref_mask > 0] = 2         # Reference in blue
    combined_img = nib.Nifti1Image(combined, lc_atlas_img.affine)
    
    for i, (display_mode, cut_coords, title) in enumerate([
        ('z', [-30], 'Axial'),
        ('y', [-32], 'Coronal'),
        ('x', [5], 'Sagittal')
    ]):
        display = plotting.plot_roi(
            combined_img,
            bg_img=mni_template,
            cut_coords=cut_coords,
            display_mode=display_mode,
            axes=axes[i],
            title=title,
            alpha=0.8
        )
    
    plt.suptitle('LC (yellow) and Reference Region (blue)', y=1.02)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, 'LC_and_reference_ROIs.png'), dpi=150, bbox_inches='tight')
    plt.show()

## 5. Overlay on Subject Data

Show the LC atlas overlaid on a registered subject's contrast maps.

In [ ]:
# Find processed subjects
processed_subjects = [d for d in os.listdir(OUTPUT_DIR) if d.startswith('sub-') and os.path.isdir(os.path.join(OUTPUT_DIR, d))]

if processed_subjects and lc_atlas_img is not None:
    sub_id = processed_subjects[0]
    sub_dir = os.path.join(OUTPUT_DIR, sub_id)
    
    # Find available contrasts
    contrasts_available = []
    for f in os.listdir(sub_dir):
        if f.endswith('_MNI.nii.gz'):
            contrast = f.replace(f'{sub_id}_', '').replace('_MNI.nii.gz', '')
            contrasts_available.append((contrast, os.path.join(sub_dir, f)))
    
    print(f"Available contrasts for {sub_id}: {[c[0] for c in contrasts_available]}")
    
    # Plot LC overlay on each contrast
    n_contrasts = len(contrasts_available)
    if n_contrasts > 0:
        fig, axes = plt.subplots(1, n_contrasts, figsize=(5*n_contrasts, 5))
        if n_contrasts == 1:
            axes = [axes]
        
        for ax, (contrast_name, contrast_path) in zip(axes, contrasts_available):
            contrast_img = nib.load(contrast_path)
            
            display = plotting.plot_roi(
                lc_atlas_img,
                bg_img=contrast_img,
                cut_coords=[-30],  # Axial at pons
                display_mode='z',
                axes=ax,
                title=contrast_name,
                alpha=0.5,
                cmap='autumn'
            )
        
        plt.suptitle(f'{sub_id} - LC Atlas Overlay on Different Contrasts', y=1.02)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'{sub_id}_LC_overlay_all_contrasts.png'), dpi=150, bbox_inches='tight')
        plt.show()
else:
    if not processed_subjects:
        print("No processed subjects found. Run notebook 02 first.")
    if lc_atlas_img is None:
        print("LC atlas not loaded.")

## 6. Summary

This notebook:
1. Loaded the 7T LC probabilistic atlas
2. Examined ROI size at different probability thresholds
3. Created a pontine tegmentum reference region for CNR calculation
4. Visualized both ROIs on subject data

**Next**: Extract signal from LC region and compute CNR (Notebook 04)